In [1]:
import os

In [2]:
liss = [1,2,3,4,5]
for i in liss[:5]:
    print(i)

1
2
3
4
5


In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

k=1
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        
        self.layers+=[nn.Sequential(nn.Conv2d(3, 16*k,  kernel_size=3) , nn.BatchNorm2d(16*k),
                      nn.ReLU(inplace=True)), nn.Sequential(nn.Conv2d(16*k, 16*k,  kernel_size=3) , nn.BatchNorm2d(16*k),
                      nn.ReLU(inplace=True))] 
        self.layers+=[nn.Sequential(nn.Conv2d(16*k, 32*k,  kernel_size=3, stride=2),  nn.BatchNorm2d(32*k),
                      nn.ReLU(inplace=True))]
        self.layers+=[nn.Sequential(nn.Conv2d(32*k, 32*k,  kernel_size=3),  nn.BatchNorm2d(32*k),
              nn.ReLU(inplace=True)),nn.Sequential(nn.Conv2d(32*k, 32*k,  kernel_size=3),  nn.BatchNorm2d(32*k),
              nn.ReLU(inplace=True))]
        self.layers+=[nn.Sequential(nn.Conv2d(32*k, 64*k,  kernel_size=3, stride=2), nn.BatchNorm2d(64*k),
                      nn.ReLU(inplace=True))]
        self.layers+=[nn.Sequential(nn.Conv2d(64*k, 64*k,  kernel_size=3, padding='valid'), nn.BatchNorm2d(64*k),
                      nn.ReLU(inplace=True))]
        self.layers+=[nn.Sequential(nn.Conv2d(64*k, 64*k,  kernel_size=1), nn.BatchNorm2d(64*k),
                      nn.ReLU(inplace=True))]
        self.layers+= [nn.AdaptiveAvgPool2d((1,1))]
        self.fc = nn.Linear(64*k, 10)
    
    def forward(self, x, acts_only=False,all_act=False):
        all_acts = []
        for i in range(len(self.layers)):
          all_acts.append(x)
          x = self.layers[i](x)

        x = self.fc(x.view(-1, 64*k))

        if all_act:
          return all_acts, x
        return x

In [3]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)
# base_path = '/content/drive/MyDrive/CKA_proj/'
# !ls '/content/drive/MyDrive/CKA_proj/net_kornblith'

In [4]:
#Save the Random 
model=Net()
prefix='net_kornblith'
PATH = prefix
if not os.path.isdir(PATH): os.makedirs(PATH)
torch.save(model.state_dict(), PATH+'/random_1.t7')

model=Net()
prefix='net_kornblith'
PATH = prefix
if not os.path.isdir(PATH): os.makedirs(PATH)
torch.save(model.state_dict(), PATH+'/random_2.t7')

In [5]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
      
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset
import copy
  
from torchvision import datasets, transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) 
transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
cifar_data_test = datasets.CIFAR10(root='.',train=False, transform=transform_val, download=True)

# Extract a subset of 100 (class balanced) samples per class

accs = []

modelsA = []
modelsB = []


print(len(cifar_data.targets))
for seed in [1]:
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 5000))
  indx_train1 = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:5000]] for classe in range(0, 10)])


  indx_train2 = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:5000]] for classe in range(0, 10)])

 # cifar_data.targets = cifar_data.targets[::-1]
  train_data1 = Subset(cifar_data, indx_train1)
  train_data2 = Subset(cifar_data, indx_train2)

  print('Num Samples For Training %d '%(train_data1.indices.shape[0]))
  
  train_loader1 = torch.utils.data.DataLoader(train_data1,
                                             batch_size=128, 
                                             shuffle=True)
  train_loader2 = torch.utils.data.DataLoader(train_data2,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader1 = torch.utils.data.DataLoader(cifar_data_test, batch_size=128, 
                                           shuffle=False)
  val_loader2 = torch.utils.data.DataLoader(cifar_data_test,
                                           batch_size=128, 
                                           shuffle=False)

  model = Net()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-3)

  for epoch in range(100):
    if epoch == 70:
      optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-4)
    train(model, device, train_loader1, optimizer, epoch, display=epoch%5==0)
    if epoch %10 == 0:
      test(model, device, val_loader1)
      
  accs.append(test(model, device, val_loader1))
  modelsA.append(copy.deepcopy(model))

  # reset everything
  model = Net()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-3)

  for epoch in range(100):
    if epoch == 70:
      optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-4)
    train(model, device, train_loader2, optimizer, epoch, display=epoch%5==0)
  modelsB.append(copy.deepcopy(model))
    
  accs.append(test(model, device, val_loader2))
accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))


Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified
50000
Num Samples For Training 50000 
Train Epoch: 0 [31200/50000 (100%)]	Loss: 1.376610


/home/mila/h/horoiste/main_env/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.1872, Accuracy: 5713/10000 (57.13%)

Train Epoch: 5 [31200/50000 (100%)]	Loss: 0.858008
Train Epoch: 10 [31200/50000 (100%)]	Loss: 0.805167

Test set: Average loss: 0.6601, Accuracy: 7677/10000 (76.77%)

Train Epoch: 15 [31200/50000 (100%)]	Loss: 0.624956
Train Epoch: 20 [31200/50000 (100%)]	Loss: 0.518202

Test set: Average loss: 0.6041, Accuracy: 7932/10000 (79.32%)

Train Epoch: 25 [31200/50000 (100%)]	Loss: 0.509710
Train Epoch: 30 [31200/50000 (100%)]	Loss: 0.494770

Test set: Average loss: 0.5610, Accuracy: 8087/10000 (80.87%)

Train Epoch: 35 [31200/50000 (100%)]	Loss: 0.663310
Train Epoch: 40 [31200/50000 (100%)]	Loss: 0.385137

Test set: Average loss: 0.5574, Accuracy: 8159/10000 (81.59%)

Train Epoch: 45 [31200/50000 (100%)]	Loss: 0.496582
Train Epoch: 50 [31200/50000 (100%)]	Loss: 0.324367

Test set: Average loss: 0.5273, Accuracy: 8257/10000 (82.57%)

Train Epoch: 55 [31200/50000 (100%)]	Loss: 0.464352
Train Epoch: 60 [31200/50000 (100%)]	Loss: 0.

In [ ]:
#Save the Random 
model=Net()
prefix='net_kornblith'
PATH = prefix
torch.save(modelsA[0].state_dict(), PATH+'/all_1.t7')

model=Net()
prefix='net_kornblith'
PATH = prefix
torch.save(modelsB[0].state_dict(), PATH+'/all_2.t7')

Train the "memorization"

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset
import copy
  
from torchvision import datasets, transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) 
transform_train = transforms.Compose([ #need no data aug
            transforms.ToTensor(),
            normalize,
        ]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
cifar_data_test = datasets.CIFAR10(root='.',train=False, transform=transform_val, download=True)

# Extract a subset of 100 (class balanced) samples per class

accs = []

modelsA = []
modelsB = []


print(len(cifar_data.targets))
for seed in [1]:
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 5000))
  indx_train1 = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:2500]] for classe in range(0, 10)])


  indx_train2 = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[2500:5000]] for classe in range(0, 10)])

  cifar_data.targets = cifar_data.targets[::-1] #randomize labels
  train_data1 = Subset(cifar_data, indx_train1)
  train_data2 = Subset(cifar_data, indx_train2)

  print('Num Samples For Training %d '%(train_data1.indices.shape[0]))
  
  train_loader1 = torch.utils.data.DataLoader(train_data1,
                                             batch_size=256, 
                                             shuffle=True)
  train_loader2 = torch.utils.data.DataLoader(train_data2,
                                             batch_size=256, 
                                             shuffle=True)

  val_loader1 = torch.utils.data.DataLoader(cifar_data_test, batch_size=128, 
                                           shuffle=False)
  val_loader2 = torch.utils.data.DataLoader(cifar_data_test,
                                           batch_size=128, 
                                           shuffle=False)

  model = Net()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-4)

  for epoch in range(100):
    train(model, device, train_loader1, optimizer, epoch, display=epoch%5==0)
    if epoch %10 == 0:
      test(model, device, val_loader1)
      
  accs.append(test(model, device, val_loader1))
  modelsA.append(copy.deepcopy(model))

  # reset everything
  model = Net()
  model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-4)

  for epoch in range(100):
    train(model, device, train_loader2, optimizer, epoch, display=epoch%5==0)
  modelsB.append(copy.deepcopy(model))
    
  accs.append(test(model, device, val_loader2))
accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))

In [ ]:
#Save the Random 
model=Net()
prefix='net_kornblith'
PATH = prefix
torch.save(modelsA[0].state_dict(), PATH+'/mem_1.t7')

model=Net()
prefix='net_kornblith'
PATH = prefix
torch.save(modelsB[0].state_dict(), PATH+'/mem_2.t7')